In [1]:
# --- load libraries ---
from semanticgis.abstract import Pipeline
from semanticgis.compilers import mermaid, qgis_recipy, validate
from IPython.display import display, Markdown

# --- Define Pipeline ---
p = Pipeline(name="Pipeline Demo")

## Notebook Overview
This notebook demonstrates how to import `semanticgis.abstract.Pipeline`, register semantic inputs, and compile the resulting DAG into Mermaid and QGIS documentation. Always run Cell 1 first to ensure the library is imported before executing the remaining steps.

## 1.  Declare input and  Milestone Data -

In [7]:
p.io.declare_input(
    name="Europa_countries",
    source="https://raw.githubusercontent.com/Esbern/gis_demo_data/main/europa.geojson",
    data_model="vector",
    measurement_scale="ratio",
    nature="discrete",
    description="European sovereign boundaries"
)


## 2. Declare the processing steps

In [8]:
# Isolate the Kingdom of Denmark
dk_handle = p.extraction.filter_by_sql(
    dataset="Europa_countries",
    where_clause="SOVEREIGNT = 'Denmark'",
    label="Kingdom of Denmark"
)

# Buffer to find the 12 NM maritime boundary
p.proximity.buffer(
    dataset=dk_handle,
    distance=0.22,  # approximately 12 nautical miles in degrees
    units="degrees",
    output_name="Danish territorial sea",
    label="12 NM territorial buffer"
)

# Buffer to find the 59 NM maritime boundary
p.proximity.buffer(
    dataset=dk_handle,
    distance=0.59,  # approximately 59 nautical miles in degrees
    units="degrees",
    output_name="Danish intrest zone sea",
    label="59 NM intrest zome"
)

## 3. Visualize and Validate the Plan --

In [6]:
display(Markdown(mermaid.compile(p)))

```mermaid
flowchart TD

    subgraph Data Sets
        node_1_data((<b>Europa_countries</b><br/>Declare Europa_countries<br/><small>vector / ratio / discrete</small>))
        node_2_data((Kingdom of Denmark<br/><small>vector / ratio / discrete</small>))
        node_3_data((<b>Danish territorial sea</b><br/>12 NM territorial buffer<br/><small>vector / ratio / discrete</small>))
        node_4_data((Kingdom of Denmark<br/><small>vector / ratio / discrete</small>))
        node_5_data((<b>Danish territorial sea</b><br/>12 NM territorial buffer<br/><small>vector / ratio / discrete</small>))
        node_6_data((<b>Danish intrest zone sea</b><br/>59 NM intrest zome<br/><small>vector / ratio / discrete</small>))
    end

    subgraph Processes
        node_2_proc[Kingdom of Denmark<br/><small>Extraction</small>]
        node_3_proc[12 NM territorial buffer<br/><small>Proximity</small>]
        node_4_proc[Kingdom of Denmark<br/><small>Extraction</small>]
        node_5_proc[12 NM territorial buffer<br/><small>Proximity</small>]
        node_6_proc[59 NM intrest zome<br/><small>Proximity</small>]
    end

    %% -- Connections --
    node_1_data --> node_2_proc
    node_2_data --> node_3_proc
    node_1_data --> node_4_proc
    node_4_data --> node_5_proc
    node_4_data --> node_6_proc
    node_2_proc --> node_2_data
    node_3_proc --> node_3_data
    node_4_proc --> node_4_data
    node_5_proc --> node_5_data
    node_6_proc --> node_6_data
```

## 4. Display a QGIS recipy

In [ ]:
display(Markdown(qgis_recipy.compile(p)))

## 5. Validate the pipeline semantics

In [ ]:
validate.compile(p)

## 6. Execution placeholder
The Python executor is paused during the functional-complex refactor. Once it returns, this section will demonstrate how to materialise the territorial sea features and render them in Folium.